In [32]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, unix_timestamp, hour, to_date
import logging
from io import StringIO, BytesIO
from minio import Minio
from minio.error import S3Error
import datetime

In [ ]:

HIVE_URI = 'thrift://hive-metastore:9083'
MINIO_ACCESS_KEY = 'FrmF5fXO0bxpBepjVUSX'
MINIO_SECRET_KEY = '5RgPTIToiUPa16HAWnRv3KcsE7y21Oo3RPA3QXTb'
MINIO_ENDPOINT = "minio:9000"
MINIO_BUCKET = "logs"


spark = SparkSession.builder \
    .appName("IcebergAsDefaultCatalog") \
    .config('spark.jars.packages', 
            'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,'
            'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,'
            'software.amazon.awssdk:bundle:2.17.178,'
            'software.amazon.awssdk:url-connection-client:2.17.178') \
    .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.spark_catalog.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://warehouse/") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.hive.metastore.sasl.enabled", "false") \
    .getOrCreate()

## Start Spark Session
print("Spark Running")


Spark Running


In [34]:
# In-memory log buffer
log_buffer = StringIO()

# Configure logger
log_handler = logging.StreamHandler(log_buffer)
log_handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)s: %(message)s'))

logger = logging.getLogger("BronzeLoader")
logger.setLevel(logging.INFO)
logger.addHandler(log_handler)
logger.addHandler(logging.StreamHandler())  # Optional: Also log to stdout

In [35]:
# Upload logs from buffer to MinIO
def flush_logs_to_minio(object_name):
    try:
        client = Minio(
            MINIO_ENDPOINT,
            access_key=MINIO_ACCESS_KEY,
            secret_key=MINIO_SECRET_KEY,
            secure=False  # Set to True if using HTTPS
        )
        if not client.bucket_exists(MINIO_BUCKET):
            client.make_bucket(MINIO_BUCKET)

        content = log_buffer.getvalue().encode("utf-8")
        client.put_object(
            MINIO_BUCKET,
            object_name,
            data=BytesIO(content),
            length=len(content),
            content_type='text/plain'
        )
        logger.info(f"Uploaded log to MinIO at {MINIO_BUCKET}/{object_name}")
        
        # Clear the buffer after upload
        log_buffer.truncate(0)
        log_buffer.seek(0)
        
    except S3Error as e:
        logger.error(f"Failed to upload log: {e}")

In [ ]:
def load_bronze(file_name, table_name):
    try:
        start_time = datetime.datetime.now()
        logger.info(f"Starting load for file: {file_name} into table: {table_name}")
        logger.info(f"Starting load for file: {file_name}")
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load(f"s3a://test/bronze.{file_name}")
        
        logger.info(f"Loaded {df.count()} records from {file_name}")
        
        df.write.format("iceberg") \
            .mode("overwrite") \
            .insertInto(f"spark_catalog.bronze.{table_name}")
        
        logger.info(f"Data written to iceberg table: {table_name}")
        end_time = datetime.datetime.now()
        elapsed_time = str(end_time - start_time)
        logger.info(f"Load completed for file: {file_name} into table: {table_name} in {elapsed_time} seconds")


    except Exception as e:
        logger.error(f"Failed to load file {file_name}: {e}")
        raise ValueError
    
    finally:
        flush_logs_to_minio(f"bronze_logs/{file_name}.log")

In [38]:
# List of files and corresponding table names
files_to_load = [
    ("crm_cust_info.csv", "crm_cust_info"),
    ("crm_prd_info.csv", "crm_prd_info"),
    ("crm_sales_details.csv", "crm_sales_details"),
    ("erp_cust_az12.csv", "erp_cust_az12"),
    ("erp_loc_a101.csv", "erp_loc_a101"),
    ("erp_px_cat_g1v2.csv", "erp_px_cat_g1v2")
]

# Load each file into the corresponding Iceberg table
for file_name, table_name in files_to_load:
    load_bronze(file_name, table_name)

# Stop the Spark session
spark.stop()

print("Data loaded into Iceberg tables successfully.")
# Note: The above code assumes that the Iceberg tables and the S3 bucket structure already exist.

Starting load for file: crm_cust_info.csv
Loaded 18493 records from crm_cust_info.csv
Data written to iceberg table: crm_cust_info
Uploaded log to MinIO at logs/bronze_logs/crm_cust_info.csv.log
Starting load for file: crm_prd_info.csv
Loaded 397 records from crm_prd_info.csv
Data written to iceberg table: crm_prd_info
Uploaded log to MinIO at logs/bronze_logs/crm_prd_info.csv.log
Starting load for file: crm_sales_details.csv
Loaded 60398 records from crm_sales_details.csv
Data written to iceberg table: crm_sales_details
Uploaded log to MinIO at logs/bronze_logs/crm_sales_details.csv.log
Starting load for file: erp_cust_az12.csv
Loaded 18483 records from erp_cust_az12.csv
Data written to iceberg table: erp_cust_az12
Uploaded log to MinIO at logs/bronze_logs/erp_cust_az12.csv.log
Starting load for file: erp_loc_a101.csv
Loaded 18484 records from erp_loc_a101.csv
Data written to iceberg table: erp_loc_a101
Uploaded log to MinIO at logs/bronze_logs/erp_loc_a101.csv.log
Starting load for f

Data loaded into Iceberg tables successfully.
